In [4]:
import torch
import torchvision
from torch import nn
import numpy as np
from torch.utils.data import Dataset, DataLoader
import matplotlib.pyplot as plt
import seaborn as sns


In [5]:
device = 'cuda'
data_folder = '~/data/FMNIST'

In [6]:
fmnist = torchvision.datasets.FashionMNIST(data_folder, download=True, \
 train=True)

In [7]:
tr_images = fmnist.data
tr_targets = fmnist.targets

In [8]:
class FMNISTDataset(Dataset):
    def __init__(self, x, y):
        x = x.float()
        x = x.view(-1, 28 * 28)
        self.x , self.y = x, y
    def __getitem__(self, index):
        x, y = self.x[index], self.y[index]
        return x.to(device), y.to(device)
    def __len__(self):
        return len(self.x)
        

In [9]:
def get_data():
    train = FMNISTDataset(tr_images, tr_targets)